<a href="https://colab.research.google.com/github/adeguindo/cloud-masking-sentinel2/blob/master/ECAAS_CommCare_Polygons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

Importing the libraries

In [ ]:
import pandas as pd
from ast import literal_eval
from shapely import geometry
import geopandas as gpd
import math
import os
import pyproj
import numpy as np

Creating paths to where the datasets will be accessed and stored

In [ ]:
data_path = "drive/My Drive/Kory/ecaas"

In [ ]:
df = pd.read_csv(data_path + '/fispthird.csv' )

When importing data, some data points will be missing. This code fills this missing gaps with NA

In [ ]:
df.fillna('None', inplace=True)

In [ ]:
df.head()

,boxes_case_id,boxes_box1_crop_condition,boxes_box1_gps,boxes_box2_gps,boxes_crop,boxes_expected_harvest_date,boxes_intercropping,boxes_main_crop_vertex1_gps,boxes_main_crop_vertex2_gps,boxes_main_crop_vertex3_gps,boxes_main_crop_vertex4_gps,boxes_main_crop_vertex5_gps,boxes_main_crop_vertex6_gps,boxes_planting_date,cce_adm1,cce_adm2,compliant,final_dry_kgs_crop_box1,final_dry_kgs_crop_box2,Unnamed: 19
0,e761656d-e99e-453d-a52a-5746a368a9b4,below_average,-11.4455056 28.9770231 1305.85 4.28,-11.4455056 28.9770231 1305.85 4.28,groundnuts,44691,yes,-11.4454282 28.9772222 1305.16 8.57,-11.4454201 28.9772197 1304.74 5.09,-11.4454212 28.9772238 1303.99 6.43,-11.4454194 28.9772224 1304.13 9.64,None,None,44539,Milenge,Kabange,Yes,0.455,0.700,0.650000
1,4c29d1dc-adca-4f65-9980-cb6ca4aba585,below_average,-16.2936397 22.0972887 1044.9 4.9,-16.2936397 22.0972887 1044.9 4.9,maize,44696,yes,-16.2936583 22.0976417 1043.8 3.96,-16.2927936 22.0980793 1041.9 4.57,-16.2937984 22.0992376 1041.3 4.09,-16.2938017 22.09924 1041.3 4.17,None,None,44557,Shangombo,Shangombo,Yes,0.650,1.000,0.650000
2,66fb3c19-faf1-4738-b4db-f2296bed97d6,average,-11.6466034 29.4635679 1182.7 4.8,-11.6466034 29.4635679 1182.7 4.8,groundnuts,44746,no,-11.6465217 29.463595 1192.2 3.9,-11.6465141 29.4635352 1192.09 4.4,-11.6465433 29.46353 1192.09 4.09,-11.646551 29.4635482 1192.09 4.0,None,None,44169,Samfya,Mungulube,Yes,0.975,1.500,0.650000
3,c6fa9d2a-8323-4a85-83c9-9ba2eb9da787,below_average,-15.1520267 26.869465 1082.0 4.7,-15.1520267 26.869465 1082.0 4.7,groundnuts,44683,no,-15.1517438 26.8695498 1065.59 5.0,-15.1516837 26.8695881 1065.5 4.59,-15.1516553 26.8695982 1065.5 4.8,-15.1509643 26.8698746 1064.4 4.59,None,None,44171,Mumbwa,Chibuluma,Yes,1.570,2.415,0.650104
4,b1cab33e-713f-435d-b271-0bf78c9b6821,good,-9.0039483 31.0611267 1450.2 5.0,-9.0039483 31.0611267 1450.2 5.0,maize,44742,no,-9.00397 31.0610117 1450.2 3.9,-9.0038198 31.0615153 1450.5 4.66,-9.0039832 31.0615758 1455.1 3.3,-9.004125 31.06161 1455.1 4.59,None,None,44528,Mpulungu,Vyamba,Yes,22.410,34.470,0.650131


This next code block creates a polygon from the individual coordinate points that were captured from the vertices of the farm. It also creates extra columns that will be used to flag out erroneous polygons.

boxes_main_crop_vertex"X"_gps - this represents the vertex number. Depends on the number of vertices that the farm has.
boxes_box1_gps - Represents the coordinate of the first box
boxes_box2_gps - Represents the coordinate of the second box

These column names might change according to your naming convention.

In [ ]:
#Creating the new variables for the extra columns that will be created.
df1 = df
geom = []
validity = []
area_poly = []
box1gps =[]
box2gps =[]
box1tobox2dist =[]
box1toCentdist =[]
box2toCentdist = []
is_within1 =[]
is_within2 =[]

for i, row in enumerate(df1.itertuples(), 1):
  # print(i)
  vertices = []

  if row.boxes_main_crop_vertex1_gps != "None":
    line = row.boxes_main_crop_vertex1_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
    # print(holder)
  if row.boxes_main_crop_vertex2_gps != "None":
    line = row.boxes_main_crop_vertex2_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
    # print(holder2)
  if row.boxes_main_crop_vertex3_gps != "None":
    line = row.boxes_main_crop_vertex3_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
    # print(holder)
  if row.boxes_main_crop_vertex4_gps != "None":
    line = row.boxes_main_crop_vertex4_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
    #print(holder)
    #Vertex 5
  if row.boxes_main_crop_vertex5_gps != "None":
    line = row.boxes_main_crop_vertex5_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
  #Vertex 6
  if row.boxes_main_crop_vertex6_gps != "None":
    line = row.boxes_main_crop_vertex6_gps.split()
    holder = [literal_eval(line[1]),literal_eval(line[0])]
    vertices.append(holder)
  print(vertices)
  #  #Vertex 7
  # if row.main_crop_vertex7_gps != "None":
  #   line = row.main_crop_vertex7_gps.split()
  #   holder = [literal_eval(line[1]),literal_eval(line[0])]
  #   vertices.append(holder)

  if len(vertices) > 0:
    vertices.append(vertices[0])
    #Create the Polygon using the Shapely package
    vertices = geometry.Polygon(vertices)
    cen = vertices.centroid
    #Checking the validity of the Polygons
    validity.append(vertices.is_valid)
    geom.append(vertices)

  print(vertices)

  #Create box1 Point Coordinates
  if row.boxes_box1_gps != 'None':
    box1 = row.boxes_box1_gps.split()
    holder = [literal_eval(box1[1]),literal_eval(box1[0])]
    box1Coord = geometry.Point(holder)
    box1gps.append(box1Coord)
    #print(box1Coord.within(vertices))
  else:
    box1Coord = (float("nan"))
    box1gps.append(box1Coord)

   #Create box2 Point Coordinates
  if row.boxes_box2_gps != 'None':
    box2 = row.boxes_box2_gps.split()
    holder = [literal_eval(box2[1]),literal_eval(box2[0])]
    box2Coord = geometry.Point(holder)

    box2gps.append(box2Coord)
  else:
    box2Coord = (float("nan"))
    box2gps.append(box2Coord)


  #Distance between one box and the other
  if ((pd.isnull(box2Coord)) or (pd.isnull(box1Coord))):
    box1tobox2dist.append((float("nan")))
  else:
    geod = pyproj.Geod(ellps='WGS84')
    angle1,angle2,distance = geod.inv(box1Coord.x, box1Coord.y, box2Coord.x, box2Coord.y)
    box1tobox2dist.append(distance)

  #box 1 to centroid distance
  if (pd.isnull(box1Coord)):
    box1toCentdist.append((float("nan")))
  else:
    angle1,angle2,distance = geod.inv(cen.x, cen.y, box1Coord.x, box1Coord.y)
    box1toCentdist.append(distance)

  #box 2 to centroid distance
  if (pd.isnull(box2Coord)):
    box2toCentdist.append((float("nan")))
  else:
    angle1,angle2,distance = geod.inv(cen.x, cen.y, box2Coord.x, box2Coord.y)
    box2toCentdist.append(distance)

  #area of the polygon
  print(vertices)
  area = abs(geod.geometry_area_perimeter(vertices)[0])
  area_poly.append(area)

#check whether box1 is inisde the polygon
  if (pd.isnull(box1Coord)):
    is_within1.append((float("nan")))
  else:
    within = box1Coord.within(vertices)
    is_within1.append(within)

  #check whether box2 is inisde the polygon
  if (pd.isnull(box2Coord)):
    is_within2.append((float("nan")))
  else:
    within = box2Coord.within(vertices)
    is_within2.append(within)





In [ ]:
print(is_within1[0:10])

[False, False, False, False, True, False, False, False, False, False]


In [ ]:
len(box1gps)


6999

In [ ]:
df1['geom'] = geom
df1['validity'] = validity
df1['area_poly'] = area_poly
df1['box1gps'] = box1gps
df1['box2gps'] = box2gps
df1['box1tobox2dist'] = box1tobox2dist
df1['box1toCentdist'] = box1toCentdist
df1['box2toCentdist'] = box2toCentdist
df1['is_within1'] = is_within1
df1['is_within2'] = is_within2


In [ ]:
df1.columns.values

array(['boxes_case_id', 'boxes_box1_crop_condition', 'boxes_box1_gps',
       'boxes_box2_gps', 'boxes_crop', 'boxes_expected_harvest_date',
       'boxes_intercropping', 'boxes_main_crop_vertex1_gps',
       'boxes_main_crop_vertex2_gps', 'boxes_main_crop_vertex3_gps',
       'boxes_main_crop_vertex4_gps', 'boxes_main_crop_vertex5_gps',
       'boxes_main_crop_vertex6_gps', 'boxes_planting_date', 'cce_adm1',
       'cce_adm2', 'compliant', 'final_dry_kgs_crop_box1',
       'final_dry_kgs_crop_box2', 'Unnamed: 19', 'geom', 'validity',
       'area_poly', 'box1gps', 'box2gps', 'box1tobox2dist',
       'box1toCentdist', 'box2toCentdist', 'is_within1', 'is_within2'],
      dtype=object)

This next section represents filtering of the datasets that have just been created. It depends with the thresholds that you will agree for your dataset, that will fairly give you the polygons that represent the true picture of what is on the ground(Accurately editable).

Some of the metrics include the distances between the two boxes(Dictated by the CCE(crop cutting experiment) protocal), area of the polygon, whether the boxes' coordinates are within the polygon or somewhere near.

In [ ]:
df2 = df1.loc[ (df1['box1toCentdist'] < 20) & (df1['box2toCentdist'] < 20) & (df1['validity'] == True) & (df1['area_poly']> 50)]

Save the filtered dataset as a CSV format.

In [ ]:
df2.to_csv(data_path + "fisp_analyzed.csv")

In this code block section, the dataframe is changed into a geodataframe inorder to look for polygon overlaps

In [ ]:
gdf = gpd.GeoDataFrame(df2, geometry = df2.geom)

In [ ]:
gdf.head()

In [ ]:
gdf.to_csv(data_path + "/data3rd.csv")

In [ ]:

gdf.to_file(driver = 'ESRI Shapefile', filename=  data_path + "/myfile.shp")

In [ ]:
gdf.columns.values

In [ ]:
ids = []
for index, row in gdf.iterrows():

  data_temp1=gdf.loc[df.boxes_case_id!=row.boxes_case_id]
    # check if intersection occured
  # print(data_temp1)
  overlaps=data_temp1[data_temp1.geometry.overlaps(row.geometry)]['boxes_case_id'].tolist()

  if len(overlaps)>0:
      ids.append(overlaps)
      print(overlaps)
      temp_list=[]

In [ ]:

arr = np.array(ids)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [ ]:
# convert array into dataframe
DF = pd.DataFrame(arr)

# save the dataframe as a csv file(This file will contain the overlaps)
DF.to_csv(os.path.join(data_path,"data3.csv"))